In [2]:
%%capture
!pip install isodate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.6 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
google_jhan = userdata.get('google_jhan')

In [4]:
import googleapiclient.discovery
import json
from google.colab import files
import isodate  # Necesario para convertir la duración ISO 8601 a segundos

# Función para crear el servicio de YouTube
def build_youtube_service(api_key):
    return googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

# Función para convertir la duración ISO 8601 a segundos
def convert_duration_to_seconds(duration):
    try:
        parsed_duration = isodate.parse_duration(duration)
        return int(parsed_duration.total_seconds())
    except Exception as e:
        print(f"Error al convertir duración: {e}")
        return 0  # Valor por defecto si hay un error

# Función para obtener los videos de la playlist
def get_playlist_videos(youtube, playlist_id):
    videos = []
    next_page_token = None

    while True:
        try:
            # Obtener los items de la playlist
            request = youtube.playlistItems().list(
                part='snippet',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            )
            response = request.execute()

            # Procesar cada video en la playlist
            for item in response['items']:
                video_id = item['snippet']['resourceId']['videoId']

                # Obtener detalles del video (incluyendo duración)
                video_details_request = youtube.videos().list(
                    part='snippet,statistics,contentDetails',
                    id=video_id
                )
                video_details_response = video_details_request.execute()

                # Extraer información del video
                video_details = video_details_response['items'][0]
                video_title = video_details['snippet']['title']
                channel_id = video_details['snippet']['channelId']
                channel_title = video_details['snippet']['channelTitle']
                video_url = f"https://www.youtube.com/watch?v={video_id}"
                channel_url = f"https://www.youtube.com/channel/{channel_id}"
                view_count = video_details['statistics'].get('viewCount', 'N/A')
                description = video_details['snippet'].get('description', 'No description available')
                publish_date = video_details['snippet']['publishedAt']

                # Obtener la duración en formato ISO 8601 y convertirla a segundos
                duration_iso = video_details['contentDetails']['duration']
                duration_seconds = convert_duration_to_seconds(duration_iso)

                # Añadir datos a la lista de videos
                videos.append({
                    'video_id': video_id,
                    'video_title': video_title,
                    'channel_id': channel_id,
                    'channel_title': channel_title,
                    'view_count': view_count,
                    'description': description,
                    'publish_date': publish_date,
                    'duration_seconds': duration_seconds,
                    'video_url': video_url,
                    'channel_url': channel_url
                })

            # Manejar paginación
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break

        except Exception as e:
            print(f"Error al obtener los videos: {e}")
            break

    return videos

# Función para guardar los datos en un archivo JSON
def save_to_json(videos, filename):
    try:
        with open(filename, 'w') as f:
            json.dump(videos, f, indent=4)
        print(f"Datos guardados en {filename}")
    except Exception as e:
        print(f"Error al guardar los datos en JSON: {e}")

# Función principal para manejar la lógica
def main(api_key, playlist_id, output_filename, descargar=False):
    youtube = build_youtube_service(api_key)
    videos_info = get_playlist_videos(youtube, playlist_id)

    # Mostrar en consola
    for video in videos_info:
        print(f"Video ID: {video['video_id']}")
        print(f"Título: {video['video_title']}")
        print(f"ID del canal: {video['channel_id']}")
        print(f"Nombre del canal: {video['channel_title']}")
        print(f"Vistas: {video['view_count']}")
        print(f"Descripción: {video['description']}")
        print(f"Fecha de publicación: {video['publish_date']}")
        print(f"Duración (segundos): {video['duration_seconds']}")
        print(f"URL del video: {video['video_url']}")
        print(f"URL del canal: {video['channel_url']}")
        print("-" * 40)

    # Guardar en archivo JSON
    save_to_json(videos_info, output_filename)

    # Descargar el archivo si se habilita
    if descargar:
        files.download(output_filename)

# Variables de configuración
if __name__ == "__main__":
    api_key = google_jhan  # Coloca tu clave de API de YouTube aquí
    playlist_id = 'PLVIxDMy39nFlefm8Bb2zZwDYLRCqoichV'  # ID de la lista de reproducción
    output_filename = 'youtube_playlist_data.json'  # Nombre del archivo JSON
    descargar = False  # Cambia a False si no deseas descargar el archivo

    main(api_key, playlist_id, output_filename, descargar)


Video ID: yNQh6aFtVFk
Título: HOLA OTRA VEZ!! | #APOCALIPSISMINECRAFT3 | EPISODIO 1 | WILLYREX Y VEGETTA
ID del canal: UC4LHNX8d8RqnDX0OezgmCTg
Nombre del canal: TheWillyrex
Vistas: 9956438
Descripción: Canal de mi compi: https://www.youtube.com/user/vegetta777
SUSCRIBETE!! ►► http://goo.gl/Cl12A
 Like y FAVORITOS Si te ha gustado!! Ayuda =)
Descuentos en Juegos: http://goo.gl/PYp8ZX

Canal Principal: http://www.youtube.com/user/Willyrex
Twitter: https://twitter.com/#!/WillyrexYT
Facebook: http://www.facebook.com/pages/Willyrex/162597417232850
- App Para Willyrex: http://myapp.wips.com/willyrexyt-extension
- App Para TheWillyrex: http://myapp.wips.com/thewillyrexyt-extension
Fecha de publicación: 2014-07-19T21:01:03Z
Duración (segundos): 1799
URL del video: https://www.youtube.com/watch?v=yNQh6aFtVFk
URL del canal: https://www.youtube.com/channel/UC4LHNX8d8RqnDX0OezgmCTg
----------------------------------------
Video ID: 9YHIn9rYCQE
Título: ¿QUIERES CASARTE CONMIGO? | #APOCALIPSISMINEC

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

### Generar csv a partir del json

In [7]:
import pandas as pd
import json
# from google.colab import files

# Función para convertir el JSON a CSV y aplicar modificaciones
def json_to_csv(json_filename, csv_filename, candidate_name, descargar=False):
    try:
        # Leer el archivo JSON
        with open(json_filename, 'r') as f:
            data = json.load(f)

        # Convertir los datos del JSON a un DataFrame de pandas
        df = pd.DataFrame(data)

        # Eliminar la columna 'description'
        if 'description' in df.columns:
            df = df.drop(columns=['description'])

        # Renombrar columnas
        df = df.rename(columns={
            'channel_title': 'channel',
            'channel_url': 'uploader_url',
            'video_id': 'id',
            'video_url': 'url',
            'video_title': 'title',
            'duration_seconds': 'duration'
        })

        # Convertir la columna 'duration' a float
        df['duration'] = df['duration'].astype(float)

        # Agregar la columna 'quality' con el valor 'high' si no existe
        if 'quality' not in df.columns:
            df['quality'] = 'high'

        # Añadir la columna 'candidate_name'
        df['candidate_name'] = candidate_name

        # Mostrar el DataFrame en consola con formato
        print("\n--- Datos del CSV ---")
        print(df.to_string(index=False))
        print("\n--- Fin de los datos ---")

        # Guardar el DataFrame como CSV
        df.to_csv(csv_filename, index=False)
        print(f"\nArchivo CSV guardado como {csv_filename}")

        # Descargar el archivo si se habilita
        if descargar:
            files.download(csv_filename)

    except Exception as e:
        print(f"Error al convertir JSON a CSV: {e}")

# Función principal para manejar la lógica
def main(json_filename, csv_filename, candidate_name, descargar=False):
    json_to_csv(json_filename, csv_filename, candidate_name, descargar)

# Variables de configuración
if __name__ == "__main__":
    json_filename = '../data/dataset.json'  # Nombre del archivo JSON
    csv_filename = '../data/dataset.csv'    # Nombre del archivo CSV de salida
    candidate_name = 'Vegetta777'                 # Nombre del candidato
    descargar = False                             # Cambia a False si no deseas descargar el archivo

    main(json_filename, csv_filename, candidate_name, descargar)



--- Datos del CSV ---
         id                                                                                               title               channel_id     channel view_count         publish_date  duration                                         url                                             uploader_url quality candidate_name
yNQh6aFtVFk                          HOLA OTRA VEZ!! | #APOCALIPSISMINECRAFT3 | EPISODIO 1 | WILLYREX Y VEGETTA UC4LHNX8d8RqnDX0OezgmCTg TheWillyrex    9956438 2014-07-19T21:01:03Z    1799.0 https://www.youtube.com/watch?v=yNQh6aFtVFk https://www.youtube.com/channel/UC4LHNX8d8RqnDX0OezgmCTg    high     Vegetta777
9YHIn9rYCQE                ¿QUIERES CASARTE CONMIGO? | #APOCALIPSISMINECRAFT3 | EPISODIO 2 | WILLYREX Y VEGETTA UCam8T03EOFBsNdR0thrFHdQ  VEGETTA777    5893403 2014-07-20T21:00:03Z    1928.0 https://www.youtube.com/watch?v=9YHIn9rYCQE https://www.youtube.com/channel/UCam8T03EOFBsNdR0thrFHdQ    high     Vegetta777
OgCx-K0TJIQ                     